# Capstone Project : IBM Data Science
This notebook contains de different assignments necessaryh to for the final project

## Part 1
### Web scrapping

In [27]:
!pip install bs4

In [28]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [29]:
nyc_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
nyc_neighborhoords = []

In [30]:
data  = requests.get(nyc_url).text

In [31]:
soup = BeautifulSoup(data,"html5lib")

In [32]:
tables = soup.find_all('table')

In [33]:
for index,table in enumerate(tables):
    if ("Toronto" in str(table)):
        table_index = index
print(table_index)

0


In [34]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [35]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [36]:
display(df)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Part 2
### Location data

In [45]:
!pip install geocoder
!pip install geopy

In [52]:
#import geocoder # import geocoder
#from geopy.geocoders import Nominatim

In [53]:
#import geocoder # import geocoder
#from geopy.geocoders import Nominatim
#geolocator = Nominatim(user_agent="toronto_explorer")

# results = []


# for index, row in df.iterrows():
#         #if row[""] =="":
#         #    distance_matrix_results.append("")
#         #else:
#     location = None
#     while (location is None):
#         location = geolocator.geocode(str(row['PostalCode'])+','+str(row['Borough'])+','+str(row['Neighborhood']))
#         results.append(location)
# df["location"] = pd.Series(results)

In [17]:
df_coord = pd.read_csv("Geospatial_Coordinates.csv", delimiter=",")
df_coord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [37]:
df = df.merge(df_coord[['Postal Code','Latitude','Longitude']], how="left", left_on="PostalCode", right_on="Postal Code")
df.drop(columns="Postal Code", inplace=True)

In [39]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Part 3
### Analyse & Clusterize

In [49]:
import folium # map rendering library
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [50]:
import geocoder # import geocoder
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [51]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto